In [1]:
import numpy as np
import pandas as pd

In [2]:
from keras import layers
from keras import Input
from keras.models import Model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers
from keras import backend as K
from keras import optimizers

import tensorflow as tf

/home/rafal/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
question = np.load('question.npy')
negative = np.load('Negative.npy')
positive = np.load('Positive.npy')

In [4]:
len(negative)

8484

In [5]:
rando = np.arange(len(question))
np.random.shuffle(rando)
question,negative,positive = question[rando],negative[rando],positive[rando]

In [6]:
def triplet_loss(y_true,y_pred):
    
    
    positive_pred = y_pred[:,1]
    negative_pred = y_pred[:,0]
    
    sum_loss = 1 - positive_pred + negative_pred
    
    loss = K.maximum(sum_loss,0.0)
    
    return K.sum(loss)
    

In [7]:
def my_log_softmax(x):
    return K.log(K.softmax(x))

# SIEMASE + TRIPLET

Multi-Perspective Sentence Similarity Modeling
with Convolutional Neural Networks - part 4.3

In [8]:
left_input = Input(shape=(20,300), name = 'n')
mid_input = Input(shape=(20,300), name = 'q')
right_input = Input(shape=(20,300), name = 'p')

In [9]:
flatten = layers.Flatten()

In [10]:
#Window size  = 3
conv1_min_window_size_3 = layers.Conv1D(300,3, activation='tanh',
                                        kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,300))
min_pool_1_size_3 = layers.GlobalMaxPooling1D()

conv1_max_window_size_3 = layers.Conv1D(300,3, activation='tanh',
                                        kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,300))
max_pool_1_size_3 = layers.GlobalMaxPooling1D()

conv1_avg_window_size_3 = layers.Conv1D(300,3, activation='tanh',
                                        kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,300))
avg_pool_1_size_3 = layers.GlobalAveragePooling1D()

In [11]:
#Window size  = 2
conv1_min_window_size_2 = layers.Conv1D(300,2, activation='tanh',
                                        kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,300))
min_pool_1_size_2 = layers.GlobalMaxPooling1D()

conv1_max_window_size_2 = layers.Conv1D(300,2, activation='tanh',
                                        kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,300))
max_pool_1_size_2 = layers.GlobalMaxPooling1D()

conv1_avg_window_size_2 = layers.Conv1D(300,2, activation='tanh',
                                        kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,300))
avg_pool_1_size_2 = layers.GlobalAveragePooling1D()

In [12]:
#Window size  = Infty = 20
min_pool_1_size_20 = layers.GlobalMaxPooling1D()

max_pool_1_size_20 = layers.GlobalMaxPooling1D()

avg_pool_1_size_20 = layers.GlobalAveragePooling1D()

In [13]:
# As there is no MinPooling in Keras, I use MaxPooling from - 1 * vector

left_size3_max = max_pool_1_size_3(conv1_max_window_size_3(left_input))
left_size3_min = min_pool_1_size_3(layers.Lambda(lambda x : x * -1 )(conv1_min_window_size_3(left_input)))
left_size3_avg = avg_pool_1_size_3(conv1_avg_window_size_3(left_input))

mid_size3_max = max_pool_1_size_3(conv1_max_window_size_3(mid_input))
mid_size3_min = min_pool_1_size_3(layers.Lambda(lambda x : x * -1 )(conv1_min_window_size_3(mid_input)))
mid_size3_avg = avg_pool_1_size_3(conv1_avg_window_size_3(mid_input))

right_size3_max = max_pool_1_size_3(conv1_max_window_size_3(right_input))
right_size3_min = min_pool_1_size_3(layers.Lambda(lambda x : x * -1 )(conv1_min_window_size_3(right_input)))
right_size3_avg = avg_pool_1_size_3(conv1_avg_window_size_3(right_input))



left_size2_max  = max_pool_1_size_2(conv1_max_window_size_2(left_input))
left_size2_min = min_pool_1_size_2(layers.Lambda(lambda x : x * -1 )(conv1_min_window_size_2(left_input)))
left_size2_avg = avg_pool_1_size_2(conv1_avg_window_size_2(left_input))

mid_size2_max = max_pool_1_size_2(conv1_max_window_size_2(mid_input))
mid_size2_min = min_pool_1_size_2(layers.Lambda(lambda x : x * -1 )(conv1_min_window_size_2(mid_input)))
mid_size2_avg = avg_pool_1_size_2(conv1_avg_window_size_2(mid_input))

right_size2_max = max_pool_1_size_2(conv1_max_window_size_3(right_input))
right_size2_min = min_pool_1_size_2(layers.Lambda(lambda x : x * -1 )(conv1_min_window_size_3(right_input)))
right_size2_avg = avg_pool_1_size_2(conv1_avg_window_size_3(right_input))



left_sizeInf_max  = max_pool_1_size_20(left_input)
left_sizeInf_min = min_pool_1_size_20(layers.Lambda(lambda x : x * -1 )(left_input))
left_sizeInf_avg = avg_pool_1_size_20(left_input)

mid_sizeInf_max = max_pool_1_size_20(mid_input)
mid_sizeInf_min = min_pool_1_size_20(layers.Lambda(lambda x : x * -1 )(mid_input))
mid_sizeInf_avg = avg_pool_1_size_20(mid_input)

right_sizeInf_max = max_pool_1_size_20(right_input)
right_sizeInf_min = min_pool_1_size_20(layers.Lambda(lambda x : x * -1 )(right_input))
right_sizeInf_avg = avg_pool_1_size_20(right_input)




In [14]:
# Per-dimension filters
   
per_dimension_layers_size_2_max = \
[[layers.Conv1D(3,2, activation='tanh',kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,1))]
for i in range(300)]


per_dimension_layers_size_3_max = \
[[layers.Conv1D(3,3, activation='tanh',kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,1))]
for i in range(300)]


per_dimension_layers_size_2_min = \
[[layers.Conv1D(3,2, activation='tanh',kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,1))]
for i in range(300)]


per_dimension_layers_size_3_min = \
[[layers.Conv1D(3,3, activation='tanh',kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,1))]
for i in range(300)]
    

In [15]:
# Per dimension filters cd

 
left_single_dimension_max_size2 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_2_max[i][0](        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(left_input))
    ))) for i in range(300)]
left_single_dimension_min_size2 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_2_min[i][0](layers.Lambda(lambda x : x * -1 )(        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(left_input))
    )))) for i in range(300)]

left_single_dimension_max_size3 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_3_max[i][0](        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(left_input))
    ))) for i in range(300)]
left_single_dimension_min_size3 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_3_min[i][0](layers.Lambda(lambda x : x * -1 )(        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(left_input))
    )))) for i in range(300)]


right_single_dimension_max_size2 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_2_max[i][0](        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(right_input))
    ))) for i in range(300)]
right_single_dimension_min_size2 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_2_min[i][0](layers.Lambda(lambda x : x * -1 )(        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(right_input))
    )))) for i in range(300)]

right_single_dimension_max_size3 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_3_max[i][0](        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(right_input))
    ))) for i in range(300)]
right_single_dimension_min_size3 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_3_min[i][0](layers.Lambda(lambda x : x * -1 )(        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(right_input))
    )))) for i in range(300)]


mid_single_dimension_max_size2 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_2_max[i][0](        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(mid_input))
    ))) for i in range(300)]
mid_single_dimension_min_size2 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_2_min[i][0](layers.Lambda(lambda x : x * -1 )(        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(mid_input))
    )))) for i in range(300)]

mid_single_dimension_max_size3 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_3_max[i][0](        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(mid_input))
    ))) for i in range(300)]
mid_single_dimension_min_size3 = [layers.GlobalMaxPooling1D()(
    (per_dimension_layers_size_3_min[i][0](layers.Lambda(lambda x : x * -1 )(        
    layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],3,1]) )(layers.Lambda(lambda x : x[:,:,i])(mid_input))
    )))) for i in range(300)]

In [16]:
# Horizontal comparison, AKA Algorithm 1 - cosine distance + L2 distance


results_L = []
for gather in ['avg','max','min']:
  
    results_L1 = [layers.dot([
        layers.concatenate( [
        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('left_size2_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('left_size3_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('left_sizeInf_%s'%(gather))))      
      ]),
        layers.concatenate( [
        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_size2_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_size3_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_sizeInf_%s'%(gather))))      
      ])
    ],axes = -1,normalize=True) for i in range(300)]


    results_L2 = [layers.Lambda(lambda x : tf.norm(x,ord = 2,axis=-1,keepdims=True))
        (layers.subtract([
        layers.concatenate( [
        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('left_size2_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('left_size3_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('left_sizeInf_%s'%(gather))))      
        ]),
        layers.concatenate( [
        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_size2_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_size3_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_sizeInf_%s'%(gather))))      
      ])
    ])) for i in range(300)]


    results_L += results_L1
    results_L += results_L2

horizontal_left_compared = layers.concatenate(results_L)

del results_L
del results_L1
del results_L2

In [17]:
# Horizontal comparison, AKA Algorithm 1 - cosine distance + L2 distance


results_R = []
for gather in ['avg','max','min']:
  
    results_R1 = [layers.dot([
        layers.concatenate( [
        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('right_size2_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('right_size3_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('right_sizeInf_%s'%(gather))))      
      ]),
        layers.concatenate( [
        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_size2_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_size3_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_sizeInf_%s'%(gather))))      
      ])
    ],axes = -1,normalize=True) for i in range(300)]


    results_R2 = [layers.Lambda(lambda x : tf.norm(x,ord = 2,axis=-1,keepdims=True))
        (layers.subtract([
        layers.concatenate( [
        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('right_size2_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('right_size3_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('right_sizeInf_%s'%(gather))))      
        ]),
        layers.concatenate( [
        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_size2_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_size3_%s'%(gather)))),

        layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1]) )
        (layers.Lambda(lambda x : x[:,i])(eval('mid_sizeInf_%s'%(gather))))      
      ])
    ])) for i in range(300)]


    results_R += results_R1
    results_R += results_R2
    
horizontal_right_compared = layers.concatenate(results_R)

del results_R
del results_R1
del results_R2

In [18]:
#Vertical comparison,AKA Algorithm 2 - cosine distance + L2 distance + L1 distance

v_results_L = []
for gather in ['avg','max','min']:
    for num,s1 in enumerate(['2','3','Inf']): 
        for s2 in ['2','3','Inf'][num:]:
        
            
            v_results_L.append(layers.dot([
                    eval('left_size%s_%s'%(s1,gather)),eval('mid_size%s_%s'%(s2,gather))
                ],axes = -1,normalize=True))
            v_results_L.append(layers.Lambda(lambda x : tf.norm(x,ord = 2,axis=-1,keepdims=True))
                         (layers.subtract([
                    eval('left_size%s_%s'%(s1,gather)),eval('mid_size%s_%s'%(s2,gather))])))
            v_results_L.append(layers.Lambda(lambda x : tf.norm(x,ord = 1,axis=-1,keepdims=True))
                         (layers.subtract([
                    eval('left_size%s_%s'%(s1,gather)),eval('mid_size%s_%s'%(s2,gather))])))    

    if gather != 'avg':
        for s1 in ['2','3']:    
            
            v_results_L1 = [layers.dot([
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('left_single_dimension_%s_size%s'%(gather,s1))]),
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('mid_single_dimension_%s_size%s'%(gather,s1))])
            ],axes = -1,normalize=True) for i in range(3)]

            v_results_L2 = [layers.Lambda(lambda x : tf.norm(x,ord = 2,axis=-1,keepdims=True))
                (layers.subtract([
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('left_single_dimension_%s_size%s'%(gather,s1))]),
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('mid_single_dimension_%s_size%s'%(gather,s1))])
            ])) for i in range(3)]

            v_results_L3 = [layers.Lambda(lambda x : tf.norm(x,ord = 1,axis=-1,keepdims=True))
                (layers.subtract([
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('left_single_dimension_%s_size%s'%(gather,s1))]),
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('mid_single_dimension_%s_size%s'%(gather,s1))])
            ])) for i in range(3)]
            
            v_results_L += v_results_L1
            v_results_L += v_results_L2
            v_results_L += v_results_L3
            
vertical_left_compared = layers.concatenate(v_results_L)

del v_results_L
del v_results_L1
del v_results_L2
del v_results_L3

In [ ]:
#Vertical comparison,AKA Algorithm 2 - cosine distance + L2 distance + L1 distance

v_results_R = []
for gather in ['avg','max','min']:
    for num,s1 in enumerate(['2','3','Inf']): 
        for s2 in ['2','3','Inf'][num:]:
        
            
            v_results_R.append(layers.dot([
                    eval('right_size%s_%s'%(s1,gather)),eval('mid_size%s_%s'%(s2,gather))
                ],axes = -1,normalize=True))
            v_results_R.append(layers.Lambda(lambda x : tf.norm(x,ord = 2,axis=-1,keepdims=True))
                         (layers.subtract([
                    eval('right_size%s_%s'%(s1,gather)),eval('mid_size%s_%s'%(s2,gather))])))
            v_results_R.append(layers.Lambda(lambda x : tf.norm(x,ord = 1,axis=-1,keepdims=True))
                         (layers.subtract([
                    eval('right_size%s_%s'%(s1,gather)),eval('mid_size%s_%s'%(s2,gather))])))    

    if gather != 'avg':
        for s1 in ['2','3']:    
            
            v_results_R1 = [layers.dot([
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('right_single_dimension_%s_size%s'%(gather,s1))]),
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('mid_single_dimension_%s_size%s'%(gather,s1))])
            ],axes = -1,normalize=True) for i in range(3)]

            v_results_R2 = [layers.Lambda(lambda x : tf.norm(x,ord = 2,axis=-1,keepdims=True))
                (layers.subtract([
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('right_single_dimension_%s_size%s'%(gather,s1))]),
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('mid_single_dimension_%s_size%s'%(gather,s1))])
            ])) for i in range(3)]

            v_results_R3 = [layers.Lambda(lambda x : tf.norm(x,ord = 1,axis=-1,keepdims=True))
                (layers.subtract([
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('right_single_dimension_%s_size%s'%(gather,s1))]),
                layers.concatenate([(layers.Lambda(lambda x : K.reshape(x,[K.shape(x)[0],1])))
                                (layers.Lambda(lambda x : x[:,i])(lay))
                   for lay in eval('mid_single_dimension_%s_size%s'%(gather,s1))])
            ])) for i in range(3)]
            
            v_results_R += v_results_R1
            v_results_R += v_results_R2
            v_results_R += v_results_R3
            
vertical_right_compared = layers.concatenate(v_results_R)

del v_results_R
del v_results_R1
del v_results_R2
del v_results_R3

In [ ]:
# Add dense layers at the top of network

dens1 = layers.Dense(256,activation='tanh',kernel_regularizer=regularizers.l2(1e-4))

droprout1 = layers.Dropout(0.5)


merged_L = droprout1(dens1(layers.concatenate([horizontal_left_compared,vertical_left_compared])))
merged_R = droprout1(dens1(layers.concatenate([horizontal_right_compared,vertical_left_compared])))


dens2 = layers.Dense(256, activation='tanh',kernel_regularizer=regularizers.l2(1e-4))
dens_pred = layers.Dense(1, activation=my_log_softmax,kernel_regularizer=regularizers.l2(1e-4))

pred_L = dens_pred(dens2(merged_L))
pred_R = dens_pred(dens2(merged_R))




pred = layers.concatenate([pred_L,pred_R], axis = -1)

model = Model(inputs = [left_input, mid_input, right_input], outputs = pred)

In [ ]:
model.summary()

In [ ]:

model.compile(loss=triplet_loss, 
              optimizer=optimizers.Adam(lr=0.001,clipnorm=1.))

In [ ]:
model.fit({'n' : negative, 'q' : question, 'p' : positive},
          np.zeros(len(negative)), #anything, basicly, not importent for loss function
          epochs=10, 
          batch_size = 128,
          verbose=1)